<a href="https://colab.research.google.com/github/Cp557/concrete_crack_detection/blob/main/vgg16_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

VGG16 Model for Concrete Crack Detection
This script sets up and trains a VGG16-based model for binary classification
of concrete images (cracked vs. non-cracked).

In [1]:
import tensorflow.keras as keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping

import urllib.request
import zipfile
import os

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# URL of the zip file
url = "https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip"

urllib.request.urlretrieve(url, "concrete_data_week4.zip")

with zipfile.ZipFile("concrete_data_week4.zip", 'r') as zip_ref:
    zip_ref.extractall("concrete_data_week4")

os.remove("concrete_data_week4.zip")

print("Download and extraction complete.")

Download and extraction complete.


In [3]:
# Global constants
num_classes = 2
image_resize = 224
batch_size_training = 100
batch_size_validation = 100

In [4]:
# ImageDataGenerator for data augmentation and preprocessing
data_generator = ImageDataGenerator(
    preprocessing_function=preprocess_input,
)

# Generator for training data
train_generator = data_generator.flow_from_directory(
    '/content/concrete_data_week4/concrete_data_week4/train',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_training,
    class_mode='categorical')

# Generator for val data
validation_generator = data_generator.flow_from_directory(
    '/content/concrete_data_week4/concrete_data_week4/valid',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_validation,
    class_mode='categorical')

Found 30001 images belonging to 2 classes.
Found 9501 images belonging to 2 classes.


In [5]:
# Model architecture
model = Sequential([
    VGG16(include_top=False,
          weights='imagenet',
          input_shape=(224, 224, 3),
          pooling='avg'),
    Flatten(),
    Dense(num_classes, activation='softmax', name='predictions')
])

# Freeze the layers of the pre-trained model
model.layers[0].trainable = False

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


In [6]:
# Get steps
train_samples = train_generator.samples
validation_samples = validation_generator.samples
steps_per_epoch = train_samples // batch_size_training
validation_steps = validation_samples // batch_size_validation


In [8]:
# Fit vgg16 model
fit_history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=validation_steps,
    callbacks=[EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)],
    verbose=0,
)

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


In [9]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ vgg16 (Functional)                   │ (None, 512)                 │      14,714,688 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ predictions (Dense)                  │ (None, 2)                   │           1,026 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 14,717,768 (56.14 MB)

 Trainable params: 1,026 (4.01 KB)

 Non-trainable params: 14,714,688 (56.13 MB)

 Optimizer params: 2,054 (8.03 KB)

In [10]:
# Save vgg16 model weights
model.save_weights('/content/drive/MyDrive/Coding/concrete_crack_detection/classifier_vgg16_model.weights.h5')